In [ ]:
# File names import
from os import listdir
from os.path import isfile, join

In [ ]:
# Healthy Controls (Eyes Open)
H_EO_path = "C:/Users/xmoot/Desktop/Jupyter/Mathematical Biology/Final Project/HUMS Dataset/H/EO"
file_names_H_EO = [f for f in listdir(H_EO_path) if isfile(join(H_EO_path, f))]

# Healthy Controls (Eyes Closed)
H_EC_path = "C:/Users/xmoot/Desktop/Jupyter/Mathematical Biology/Final Project/HUMS Dataset/H/EC"
file_names_H_EC = [f for f in listdir(H_EC_path) if isfile(join(H_EC_path, f))]

# Healthy Controls (Task)
H_TASK_path = "C:/Users/xmoot/Desktop/Jupyter/Mathematical Biology/Final Project/HUMS Dataset/H/TASK"
file_names_H_TASK = [f for f in listdir(H_TASK_path) if isfile(join(H_TASK_path, f))]

# MDD (Eyes Open)
MDD_EO_path = "C:/Users/xmoot/Desktop/Jupyter/Mathematical Biology/Final Project/HUMS Dataset/MDD/EO"
file_names_MDD_EO = [f for f in listdir(MDD_EO_path) if isfile(join(MDD_EO_path, f))]

# MDD (Eyes Closed)
MDD_EC_path = "C:/Users/xmoot/Desktop/Jupyter/Mathematical Biology/Final Project/HUMS Dataset/MDD/EC"
file_names_MDD_EC = [f for f in listdir(MDD_EC_path) if isfile(join(MDD_EC_path, f))]

# MDD (Task)
MDD_TASK_path = "C:/Users/xmoot/Desktop/Jupyter/Mathematical Biology/Final Project/HUMS Dataset/MDD/TASK"
file_names_MDD_TASK = [f for f in listdir(MDD_TASK_path) if isfile(join(MDD_TASK_path, f))]

# Create file names list
file_names = [file_names_H_EO, file_names_H_EC, file_names_H_TASK, file_names_MDD_EO, file_names_MDD_EC, file_names_MDD_TASK]

In [ ]:
# Load Data
import numpy as np
import mne

H_EO_data = []
H_EC_data = []
H_TASK_data = []
MDD_EO_data = []
MDD_EC_data = []
MDD_TASK_data = []

for i in range(len(file_names)):
    for j in range(len(file_names[i])):
        if i == 0:
            file =  "C:/Users/xmoot/Desktop/Jupyter/Mathematical Biology/Final Project/HUMS Dataset/H/EO/" + file_names[i][j]
            data = mne.io.read_raw_edf(file)
            H_EO_data.append(data.get_data())
        if i == 1:
            file = "C:/Users/xmoot/Desktop/Jupyter/Mathematical Biology/Final Project/HUMS Dataset/H/EC/" + file_names[i][j]
            data = mne.io.read_raw_edf(file)
            H_EC_data.append(data.get_data())
        if i == 2:
            file = "C:/Users/xmoot/Desktop/Jupyter/Mathematical Biology/Final Project/HUMS Dataset/H/TASK/" + file_names[i][j]
            data = mne.io.read_raw_edf(file)
            H_TASK_data.append(data.get_data())
        if i == 3:
            file = "C:/Users/xmoot/Desktop/Jupyter/Mathematical Biology/Final Project/HUMS Dataset/MDD/EO/" + file_names[i][j]
            data = mne.io.read_raw_edf(file)
            MDD_EO_data.append(data.get_data())
        if i == 4:
            file = "C:/Users/xmoot/Desktop/Jupyter/Mathematical Biology/Final Project/HUMS Dataset/MDD/EC/" + file_names[i][j]
            data = mne.io.read_raw_edf(file)
            MDD_EC_data.append(data.get_data())
        if i == 5:
            file = "C:/Users/xmoot/Desktop/Jupyter/Mathematical Biology/Final Project/HUMS Dataset/MDD/TASK/" + file_names[i][j]
            data = mne.io.read_raw_edf(file)
            MDD_TASK_data.append(data.get_data())
            if j == len(file_names[5]) - 1:
                print("Finished")

In [ ]:
# Create a filter function to filter EEG signals (due to electrical noise of the power grid)
def filter(data):
    # Input: numpy ndarray
    
    # Apply a high-pass filter at 0.5 Hz
    mne.filter.filter_data(data, sfreq = 256, l_freq=0.5, h_freq=None)
    # Apply a low-pass filter at 70 Hz
    mne.filter.filter_data(data, sfreq = 256, l_freq=None, h_freq=70)
    # Apply a notch filter at 50 Hz
    mne.filter.notch_filter(data, Fs = 256, freqs=50)

# We keep only the first 19 channels, as the other 3 are redundant
# Then for each sample, we apply our filter 
for i in range(len(H_EO_data)):
    H_EO_data[i] = H_EO_data[i][0:19]
    filter(H_EO_data[i])
for i in range(len(H_EC_data)):
    H_EC_data[i] = H_EC_data[i][0:19]
    filter(H_EC_data[i])
for i in range(len(H_TASK_data)):
    H_TASK_data[i] = H_TASK_data[i][0:19]
    filter(H_TASK_data[i])
for i in range(len(MDD_EO_data)):
    MDD_EO_data[i] = MDD_EO_data[i][0:19]
    filter(MDD_EO_data[i])
for i in range(len(MDD_EC_data)):
    MDD_EC_data[i] = MDD_EC_data[i][0:19]
    filter(MDD_EC_data[i])
for i in range(len(MDD_TASK_data)):
    MDD_TASK_data[i] = MDD_TASK_data[i][0:19]
    filter(MDD_TASK_data[i])
    if i == len(MDD_TASK_data) - 1:
        print("Finished")

In [ ]:
# Do not run this
temp1 = H_TASK_data
temp2 = MDD_TASK_data

In [ ]:
# Reload
H_TASK_data = temp1
MDD_TASK_data = temp2

In [ ]:
# We will train on the TASK data (ignoring EC and EO data for now)
# Each EEG signal is varying in time length. We first find the EEG signal with least time
M_1 = min([np.shape(H_TASK_data[i])[1] for i in range(len(H_TASK_data))])
M_2 = min([np.shape(MDD_TASK_data[i])[1] for i in range(len(MDD_TASK_data))])

# Round down to nearest order of 10000
M = min(M_1, M_2) - 3856

In [ ]:
# Cut the end of each EEG signal down to this minimum time length M
# Split into signal lengths of 10000 discrete time points each
# Take the average across each time point
for i in range(len(H_TASK_data)):
    length = np.shape(H_TASK_data[i])[1]
    if length > M:
        H_TASK_data[i] = np.delete(H_TASK_data[i], list(range(M, length)), 1)
        H_TASK_data[i] = np.split(H_TASK_data[i], 15, axis = 1)

for i in range(len(MDD_TASK_data)):
    length = np.shape(MDD_TASK_data[i])[1]
    if length > M:
        MDD_TASK_data[i] = np.delete(MDD_TASK_data[i], list(range(M, length)), 1)
        MDD_TASK_data[i] = np.split(MDD_TASK_data[i], 15, axis = 1)

In [ ]:
# Dimensionality reduction
# Since there are too many time points, we must reduce the data in meaningful manner to construct our features
# We begin by averaging the data across each 10000 discrete time points 
H_TASK_avg = np.zeros((28,15,19))
MDD_TASK_avg = np.zeros((33,15,19))

for i in range(len(H_TASK_data)):
    for j in range(15):
        for k in range(19):
            H_TASK_avg[i][j][k] = np.mean(H_TASK_data[i][j][k])

for i in range(len(MDD_TASK_data)):
    for j in range(15):
        for k in range(19):
            MDD_TASK_avg[i][j][k] = np.mean(MDD_TASK_data[i][j][k])

In [ ]:
# Compile our H_TASK_data samples and MDD_TASK_data samples into an array
# 0 corresponds to Healthy Control and 1 corresponds to MDD
# We flatten our matrices according to 19 electrodes * 15 time points = 285
total_TASK_data = [[(H_TASK_avg[i].flatten()).reshape(285,1), 0] for i in range(len(H_TASK_data))]
for i in range(len(MDD_TASK_data)):
      total_TASK_data.append([(MDD_TASK_avg[i].flatten()).reshape(285,1), 1])

# Shuffle our samples, otherwise this will interfere with training
import random
random.shuffle(total_TASK_data)

In [ ]:
# We now split the data into our input array X, and output array Y
# We also to flatten each of our inputs from total_TASK_data into vectors, so we get the input matrix X = [x^(1) x^(2) . . . x^(m)]
# We have n = 285 features per input vector, so our matrix will be of size 285 x 61
input_vectors_TASK = [total_TASK_data[i][0] for i in range(len(total_TASK_data))]
X = np.dstack(input_vectors_TASK).squeeze()

output_vectors_TASK = [total_TASK_data[i][1] for i in range(len(total_TASK_data))]
Y = np.dstack(output_vectors_TASK).squeeze()
Y = Y.reshape(-1,1)

In [ ]:
# Torch
import torch			
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# Convert to tensors
X_tensor = torch.Tensor(X.T)
Y_tensor = torch.Tensor(Y)

# Datasets
# Load training set, test set, and their dataloaders
trainset = TensorDataset(X_tensor[0:40], Y_tensor[0:40])
testset = TensorDataset(X_tensor[40:61], Y_tensor[40:61] )
trainloader = DataLoader(trainset)
testloader = DataLoader(testset)
classes = (0,1)

In [ ]:
# Model (7 layer DNN)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(285, 512)
        self.fc2 = nn.Linear(512, 1024)
        self.fc3 = nn.Linear(1024, 1024)
        self.fc4 = nn.Linear(1024, 512)
        self.fc5 = nn.Linear(512, 256)
        self.fc6 = nn.Linear(256, 128)
        self.fc7 = nn.Linear(128, 64)
        self.fc8 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.sigmoid(self.fc8(x))
        return x

In [ ]:
# Neural Network
net = Net()

# Loss function (binary cross entropy)
criterion = nn.BCELoss()

# Optimizer (SGD with momentum)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# # Optimizer: Adams
# optimizer = optim.Adam(params = net.parameters(), lr = 0.0001)

# train the model
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

# save trained model
PATH = '7_DNN_1.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
# Training accuracy
# Load trained network
net = Net()
net.load_state_dict(torch.load(PATH))
							  
# Accuracy on the full training set (700 graphs)
correct = 0
total = 0
for data in trainloader:
    inputs, labels = data
    outputs = net(inputs).data
    
    # Probability threshold of 0.5
    if outputs > 0.5:
        prediction = 1
    else:
        prediction = 0
    total += 1
    if prediction == labels:
        correct += 1
print('Training accuracy of the network on the 40 patients: %d %%' % (
    100 * correct / total))

In [ ]:
# Test accuracy
# Load trained network
net = Net()
net.load_state_dict(torch.load(PATH))
							  
# Accuracy on the full test set (200 graphs)
correct = 0
total = 0
for data in testloader:
    inputs, labels = data
    outputs = net(inputs).data
    
    # Compute the prediction from the sigmoid probability
    # We set the probability threshold of 0.5
    # This threshold works because we have nearly the same number of MDD patients (n = 33) as Healthy patients (n = 28)
    if outputs > 0.5:
        prediction = 1
    else:
        prediction = 0
    total += 1
    if prediction == labels:
        correct += 1
print('Test accuracy of the network on the 20 patients: %d %%' % (
    100 * correct / total))